In [1]:
import QuantLib as ql

In [2]:
# Set Evaluation Date
today = ql.Date(31,3,2015)
ql.Settings.instance().setEvaluationDate(today)
# Setup the yield termstructure
rate = ql.SimpleQuote(0.03)
rate_handle = ql.QuoteHandle(rate)
dc = ql.Actual365Fixed()
disc_curve = ql.FlatForward(today, rate_handle, dc)
disc_curve.enableExtrapolation()
hyts = ql.YieldTermStructureHandle(disc_curve)

In [3]:
i1 = ql.EuriborSwapIsdaFixA(ql.Period("10Y"),hyts)

In [4]:
i2 = ql.EuriborSwapIsdaFixA(ql.Period("2Y"),hyts)

In [5]:
s = ql.SwapSpreadIndex("CMS 10Y-2Y", i1, i2)

In [6]:
print(s.currency())

European Euro


In [7]:
print(s.dayCounter())

30/360 (Bond Basis) day counter


In [8]:
print(s.fixingCalendar())

TARGET calendar


In [9]:
print(s.swapIndex1())

EuriborSwapIsdaFixA10Y 30/360 (Bond Basis) index


In [10]:
print(s.swapIndex2())

EuriborSwapIsdaFixA2Y 30/360 (Bond Basis) index


In [11]:
print(s.tenor())

10Y


In [12]:
print(s.fixingDate(ql.Date(10,10,2018)))

October 8th, 2018


In [13]:
s.fixing(ql.Date(10,10,2018))

-1.665979098137635e-05

In [14]:
c = ql.CmsSpreadCoupon(ql.Date(10,10,2019), 1000000, ql.Date(10,10,2018), ql.Date(10,10,2019), 2, s)

In [15]:
c.nominal()

1000000.0

In [16]:
c.fixingDays()

2

In [17]:
schedule = ql.Schedule([ql.Date(10,10,2018), ql.Date(10,10,2019)])

In [18]:
l = ql.CmsSpreadLeg([1000000], schedule, s)

In [19]:
for cf in l:
    print(cf)
    print(cf.date())
    f = ql.as_floating_rate_coupon(cf)
    print(f)

<QuantLib.QuantLib.CashFlow; proxy of <Swig Object of type 'std::vector< boost::shared_ptr< CashFlow > >::value_type *' at 0x0000017A176F8960> >
October 10th, 2019
<QuantLib.QuantLib.FloatingRateCoupon; proxy of <Swig Object of type 'FloatingRateCouponPtr *' at 0x0000017A179A6AE0> >


In [20]:
f.fixingDays()

2

In [21]:
cap = ql.CappedFlooredCmsSpreadCoupon(ql.Date(10,10,2019), 1000000, ql.Date(10,10,2018), ql.Date(10,10,2019), 2, s,
                                     1.0, 0.0, 0.0)

In [22]:
cap.cap()

0.0

In [23]:
cap.isCapped()

True

In [29]:
volQuote = ql.QuoteHandle(ql.SimpleQuote(0.2))
swaptionVol = ql.ConstantSwaptionVolatility(0, ql.TARGET(), ql.ModifiedFollowing, volQuote, ql.Actual365Fixed())
swvol_handle = ql.SwaptionVolatilityStructureHandle(swaptionVol)

In [30]:
mean_reversion = ql.QuoteHandle(ql.SimpleQuote(0.01))
cms_pricer = ql.LinearTsrPricer(swvol_handle, mean_reversion)

In [32]:
correlation = ql.QuoteHandle(ql.SimpleQuote(0.9))
cms_spread_pricer = ql.LognormalCmsSpreadPricer(cms_pricer, correlation)

In [33]:
cap.setPricer(cms_spread_pricer)

In [34]:
cap.amount()

-1821.751086788874

In [35]:
c.setPricer(cms_spread_pricer)

In [36]:
c.amount()

500.106184243513

In [41]:
cms_spread_pricer.correlation().value()

0.9

In [42]:
cms_spread_pricer.swapletPrice()

0.0004365379176095654

In [43]:
cms_spread_pricer.swapletRate()

0.000500106184243513

In [46]:
cms_spread_pricer.capletPrice(0.001)

0.0015951849277328867